In [ ]:
# scRNA analysis trial of the metabolism focused  scCellfie package
#Description: a metabolic potential analysis of the intestine - focusing no enterocytes
#Figures generated Figure 5A, Figure 5B, Figure 5C

In [ ]:
## To avoid warnings
import warnings
warnings.filterwarnings("ignore")

#Load the packages
import sccellfie
import scanpy as sc
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import glasbey

import textwrap

print(sc.__version__)
print(sccellfie.__version__)

In [ ]:
#Load the data 
# An asterisk beside the chunk means it is still running 
adata = sc.read(filename='/path/to/analysis/directory/Seurat_Files/Analysis2.7_Ileum_counts_scCellfie.h5ad')


In [ ]:
print(adata.obs.columns)

In [ ]:
#once loaded check the contents of the file
adata 
print(adata)

In [ ]:
#Specify the batch 
batch_key = 'Mouse' 

In [ ]:
#Basic scCllfie Pipeline
results = sccellfie.run_sccellfie_pipeline(adata,
                                           organism='mouse',
                                           sccellfie_data_folder=None,
                                           n_counts_col=None, # Total counts per cell will be computed if left as None,
                                           process_by_group=False, # Whether to do the processing by cell groups
                                           groupby=None, # Column indicating cell groups if `process_by_group=True`
                                           neighbors_key='neighbors', # Neighbors information if precomputed. Otherwise, it will be computed here
                                           n_neighbors=10, # Number of neighbors to use
                                           batch_key=batch_key, # The appropriate batch to correct for
                                           threshold_key='sccellfie_threshold',  # This is for using the default database. If personalized thresholds are used, specificy column name
                                           smooth_cells=True, # Whether to perform gene expression smoothing before running the tool
                                           alpha=0.33, # Importance of neighbors' expression for the smoothing (0 to 1)
                                           chunk_size=5000, # Number of chunks to run the processing steps (helps with the memory)
                                           disable_pbar=False,
                                           save_folder="/path/to/analysis/directory/scCellfie/results/", # In case results will be saved. If so, results will not be returned and should be loaded from the folder (see sccellfie.io.load_data function
                                           save_filename="scCell_fie_analyzed" # Name for saving the files, otherwise a default name will be used
                                          )

In [ ]:
#Can I view the results 
results.keys()
print(results['task_info'].head())
print(results['task_info'].columns)

In [ ]:
#Save a different way 
sccellfie.io.save_adata(adata=results['adata'], output_directory='/path/to/analysis/directory/scCellfie/results/', filename='Ileum_scCellfie')

In [ ]:
sccellfie.io.save_adata?


In [ ]:
#Save the Task_info which is not usually stored correctly
import pandas as pd

#save it explicitly:
if 'task_info' in results:
    task_info_df = results['task_info']
    task_info_df.to_csv("/path/to/analysis/directory/scCellfie/results/scCell_fie_task_info.csv")


In [ ]:
#Load results from saved 
import sccellfie.io

print(hasattr(sccellfie.io, "load_data")) # Should return True if the function exists
help(sccellfie.io.load_data)

adata = sccellfie.io.load_adata(folder = '/path/to/analysis/directory/scCellfie/results/', filename = 'Ileum_scCellfie', 
                        reactions_filename='Ileum_scCellfie_reactions', metabolic_tasks_filename='Ileum_scCellfie_metabolic_tasks',  verbose=True)

# Load the Results task_info
task_info_df = pd.read_csv("/path/to/analysis/directory/scCellfie/results/scCell_fie_task_info.csv", index_col=0)

#Assign Task_info
adata.metabolic_tasks.uns['task_info'] = task_info_df

In [ ]:
#Visualize results part 1 
cell_group = 'secondlevel'
infections = "InfectionStatus"
palette = glasbey.extend_palette('Set2',
    palette_size=max([10, adata.metabolic_tasks.obs[cell_group].nunique()])
)

In [ ]:
for task in adata.metabolic_tasks.var_names:
    print(task)

In [ ]:
#Exploration 
metabolic_tasks = ['Glycogen biosynthesis',
                   'ATP generation from glucose (hypoxic conditions) - glycolysis',
                   'Synthesis of taurine from cysteine',
                   'Synthesis of serotonin from tryptophan',
                   "Synthesis of arginine from glutamine",
                   "Reactive oxygen species detoxification (H2O2 to H2O)",
                   "Phosphatidyl-choline synthesis",
                   "Oxidative phosphorylation via succinate-coenzyme Q oxidoreductase (COMPLEX II)",
                   "O-linked glycosylation",
                   "N-linked glycosylation",
                   "Krebs cycle - NADH generation",
                   "Glycogen degradation",
                   "Mannose degradation (to fructose-6-phosphate)",
                   "Cholesterol synthesis",
                   "Calnexin/calreticulin cycle"
                  ]

plt.rcParams['figure.figsize'] = (3,3)
plt.rcParams['font.size'] = 10

sc.pl.embedding(adata.metabolic_tasks,
                color=[cell_group] + metabolic_tasks,
                ncols=1,
                palette=palette,
                frameon=False,
                basis='X_wnn.umap_cc',
                wspace=0.7,
                title=["\n".join(textwrap.wrap(t, width=60)) for t in [cell_group] + metabolic_tasks],
                cmap='OrRd'
               )

In [ ]:
fig, axes = sccellfie.plotting.create_multi_violin_plots(adata.metabolic_tasks,
                                                         features=metabolic_tasks,
                                                         groupby=cell_group, 
                                                         stripplot=False,
                                                         n_cols=2,
                                                         ylabel='Metabolic Score'
                                                        )

In [ ]:
#stacked violins
ax = sc.pl.stacked_violin(adata.metabolic_tasks, metabolic_tasks, groupby=cell_group, swap_axes=True, dendrogram=False, standard_scale='var')

In [ ]:
#Heatmap 
ax = sc.pl.heatmap(adata.metabolic_tasks, var_names=metabolic_tasks, groupby=cell_group, cmap="YlGnBu", swap_axes=True, dendrogram=True,
                   figsize=(16, 4)
                  )

In [ ]:
#Aggregate based on cell type 
agg = sccellfie.expression.aggregation.agg_expression_cells(adata.metabolic_tasks, groupby=cell_group, agg_func='trimean')

In [ ]:
#min max normalization 
input_df = sccellfie.preprocessing.matrix_utils.min_max_normalization(agg.T, axis=1)

In [ ]:
plt.figure(figsize=(16, 4))
g = sns.heatmap(input_df.loc[metabolic_tasks,:], cmap='YlGnBu', linewidths=0.5, xticklabels=1, yticklabels=1)

cbar = g.collections[0].colorbar
cbar.set_label('Scaled metabolic activity', size=14, rotation=270, labelpad=25)  # Change colorbar label size and rotation

# Uncomment code below to save figure
# plt.savefig('./figures/Heatmap-Seaborn.pdf', dpi=300, bbox_inches='tight')

In [ ]:
#Radial Plots 
df_melted = pd.melt(input_df.reset_index(), id_vars='Task', var_name='cell_type', value_name='scaled_trimean')
df_melted = df_melted.rename(columns={'Task': 'metabolic_task'})

In [ ]:
cell_types = ['Enterocytes', 'Stem Cells', 'Transit Amplifying Cells', 'Goblet Cells']

In [ ]:
# Create figure with subplots

fig = plt.figure(figsize=(16, 16))
ax1 = fig.add_subplot(221, projection='polar')
ax2 = fig.add_subplot(222, projection='polar')
ax3 = fig.add_subplot(223, projection='polar')
ax4 = fig.add_subplot(224, projection='polar')

for i, (cell, ax) in enumerate(zip(cell_types, [ax1, ax2, ax3, ax4])):
    sccellfie.plotting.create_radial_plot(df_melted,
                                          adata.metabolic_tasks.uns['task_info'],
                                          cell_type=cell,
                                          ax=ax,
                                          show_legend=i == 1,
                                          ylim=1.0,
                                          sort_by_value=True
                                          )

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Create group label for disease x celltype
adata.metabolic_tasks.obs['group_label'] = (
    adata.metabolic_tasks.obs['secondlevel'].astype(str) + "_" +
    adata.metabolic_tasks.obs['InfectionStatus'].astype(str)
)

# Aggregate using group_label
agg = sccellfie.expression.aggregation.agg_expression_cells(
    adata.metabolic_tasks,
    groupby='group_label',
    agg_func='trimean'
)

# Normalize and melt for plotting
input_df = sccellfie.preprocessing.matrix_utils.min_max_normalization(agg.T, axis=1)
df_melted = pd.melt(input_df.reset_index(), id_vars='Task', var_name='group_label', value_name='scaled_trimean')
df_melted = df_melted.rename(columns={'Task': 'metabolic_task'})


df_melted['metabolic_task'] = df_melted['metabolic_task'].astype(str).str.strip()
adata.metabolic_tasks.uns['task_info']['Task'] = adata.metabolic_tasks.uns['task_info']['Task'].astype(str).str.strip()

# Filter valid tasks
valid_tasks = set(adata.metabolic_tasks.uns['task_info']['Task'])
df_melted = df_melted[df_melted['metabolic_task'].isin(valid_tasks)].copy()

# Filter to enterocytes
enterocyte_df = df_melted[df_melted['group_label'].str.startswith("Enterocyte")].copy()

# Extract condition and cell type
enterocyte_df['condition'] = enterocyte_df['group_label'].str.split('_').str[1]
enterocyte_df['cell_type'] = enterocyte_df['group_label'].str.split('_').str[0]


conditions = enterocyte_df['condition'].unique()

n = len(conditions)
ncols = 3
nrows = (n // ncols) + int(n % ncols != 0) + 1  # +1 for legend
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, subplot_kw={'projection': 'polar'}, figsize=(5 * ncols, 5 * nrows))
axes = axes.flatten()

# Plot each condition
for i, (condition, ax) in enumerate(zip(conditions, axes)):
    condition_df = enterocyte_df[enterocyte_df['condition'] == condition]
    sccellfie.plotting.create_radial_plot(
        condition_df,
        adata.metabolic_tasks.uns['task_info'],
        ax=ax,
        show_legend=False,
        ylim=1.0,
        sort_by_value=True
    )
    ax.set_title(condition)

# Legend subplot
legend_index = len(conditions)
if legend_index < len(axes):
    legend_ax = axes[legend_index]
    legend_ax.set_axis_off()

    # Dummy plot for legend handles
    dummy_ax = plt.subplot(111, polar=True)
    condition_df = enterocyte_df[enterocyte_df['condition'] == conditions[0]]
    tmp = sccellfie.plotting.create_radial_plot(
        condition_df,
        adata.metabolic_tasks.uns['task_info'],
        ax=dummy_ax,
        show_legend=True,
        ylim=1.0,
        sort_by_value=True
    )
    handles, labels = dummy_ax.get_legend_handles_labels()
    plt.close(dummy_ax.figure)

    # Add legend
    legend_ax.legend(handles, labels, loc='center', frameon=False)
    legend_ax.set_title("Metabolic Systems")


for j in range(legend_index + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
# Save figure
# Figure 5A
output_path = "/path/to/analysis/directory/scCellfie/Images/scCellfie_radial_plots_enterocytes.svg"
fig.savefig(output_path, format="svg", dpi=300)
plt.show()
print(f"Figure saved to: {output_path}")


In [ ]:
# Check the main structure
print(adata.obs.head())

task_matrix = adata.metabolic_tasks.X  # this is usually a numpy array

# Get cell names
cells = adata.metabolic_tasks.obs_names

# Get task names
tasks = adata.metabolic_tasks.var_names

# Convert to DataFrame
tasks_df = pd.DataFrame(task_matrix, index=cells, columns=tasks)

# Preview
tasks_df.head()
tasks_df.to_csv("/path/to/analysis/directory/scCellfie/cell_task_scores.csv", index=True)

In [ ]:
# For the Nippo MEtabolism Lipids - Which of the specific pathways are the two higest 
# Filter and sort again (if not done yet)
lipid_tasks = adata.metabolic_tasks.uns['task_info']
lipid_tasks = lipid_tasks[lipid_tasks['System'].str.upper() == "LIPIDS METABOLISM"].copy()
lipid_task_names = set(lipid_tasks['Task'].str.strip())

nipp_df = enterocyte_df[
    (enterocyte_df['condition'] == "Nippostrongylus") &
    (enterocyte_df['metabolic_task'].isin(lipid_task_names))
].copy()

# Merge with task_info to get full metadata
nipp_df = nipp_df.merge(
    adata.metabolic_tasks.uns['task_info'],
    how='left',
    left_on='metabolic_task',
    right_on='Task'
)

# Sort by scaled_trimean (i.e., enrichment level)
nipp_df_sorted = nipp_df.sort_values("scaled_trimean", ascending=False)

# Select top 2
top2_lipid_tasks = nipp_df_sorted.head(5)


print(top2_lipid_tasks[['metabolic_task', 'scaled_trimean', 'System', 'Subsystem']])

In [ ]:
#Graph the total profile of each infection (All cells) - potentially biased for cell type abundances

import matplotlib.pyplot as plt

# Create group label for disease x celltype

adata.metabolic_tasks.obs['group_label'] = adata.metabolic_tasks.obs['InfectionStatus']

agg = sccellfie.expression.aggregation.agg_expression_cells(
    adata.metabolic_tasks,
    groupby='group_label',
    agg_func='trimean'
)

input_df = sccellfie.preprocessing.matrix_utils.min_max_normalization(agg.T, axis=1)
df_melted = pd.melt(input_df.reset_index(), id_vars='Task', var_name='group_label', value_name='scaled_trimean')
df_melted = df_melted.rename(columns={'Task': 'metabolic_task'})

df_melted['metabolic_task'] = df_melted['metabolic_task'].astype(str).str.strip()
adata.metabolic_tasks.uns['task_info']['Task'] = adata.metabolic_tasks.uns['task_info']['Task'].astype(str).str.strip()
valid_tasks = set(adata.metabolic_tasks.uns['task_info']['Task'])
df_melted = df_melted[df_melted['metabolic_task'].isin(valid_tasks)].copy()


#  Get unique conditions present for Enterocytes
conditions = df_melted['group_label'].unique()
df_melted['cell_type'] = 'All'
# Setup subplots
n = len(conditions)
ncols = 3
nrows = (n // ncols) + int(n % ncols != 0)
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, subplot_kw={'projection': 'polar'}, figsize=(5 * ncols, 5 * nrows))
axes = axes.flatten()

# Plot each condition's radial plot
for i, (condition, ax) in enumerate(zip(conditions, axes)):
    condition_df = df_melted[df_melted['group_label'] == condition]
    sccellfie.plotting.create_radial_plot(
        condition_df,
        adata.metabolic_tasks.uns['task_info'],
        ax=ax,
        show_legend=(i == 7),
        ylim=1.0,
        sort_by_value=True
    )
    ax.set_title(condition)

# Remove unused axes
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Create group label for disease x celltype

adata.metabolic_tasks.obs['group_label'] = (
    adata.metabolic_tasks.obs['secondlevel'].astype(str) + "_" +
    adata.metabolic_tasks.obs['InfectionStatus'].astype(str)
)

# Aggregate using group_label column name
agg = sccellfie.expression.aggregation.agg_expression_cells(
    adata.metabolic_tasks,
    groupby='group_label',  # group by this obs column
    agg_func='trimean'
)

# Normalize and melt for plotting
input_df = sccellfie.preprocessing.matrix_utils.min_max_normalization(agg.T, axis=1)
df_melted = pd.melt(input_df.reset_index(), id_vars='Task', var_name='group_label', value_name='scaled_trimean')
df_melted = df_melted.rename(columns={'Task': 'metabolic_task'})
# Make sure both task columns are strings and stripped
df_melted['metabolic_task'] = df_melted['metabolic_task'].astype(str).str.strip()
adata.metabolic_tasks.uns['task_info']['Task'] = adata.metabolic_tasks.uns['task_info']['Task'].astype(str).str.strip()

# Filter df_melted to only valid tasks present in task_info
valid_tasks = set(adata.metabolic_tasks.uns['task_info']['Task'])
df_melted = df_melted[df_melted['metabolic_task'].isin(valid_tasks)].copy()

# Filter to only enterocytes (group_label starting with "Enterocytes_")
enterocyte_df = df_melted[df_melted['group_label'].str.startswith("Enterocytes_")].copy()

# Extract condition from group_label for easier filtering and plotting
enterocyte_df['condition'] = enterocyte_df['group_label'].str.split('_').str[1]
enterocyte_df['cell_type'] = enterocyte_df['group_label'].str.split('_').str[0]

conditions = enterocyte_df['condition'].unique()


n = len(conditions)
ncols = 3
nrows = (n // ncols) + int(n % ncols != 0)
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, subplot_kw={'projection': 'polar'}, figsize=(5 * ncols, 5 * nrows))
axes = axes.flatten()

# Plot each condition's radial plot
for i, (condition, ax) in enumerate(zip(conditions, axes)):
    condition_df = enterocyte_df[enterocyte_df['condition'] == condition]
    sccellfie.plotting.create_radial_plot(
        condition_df,
        adata.metabolic_tasks.uns['task_info'],
        ax=ax,
        show_legend=(i == 0),
        ylim=1.0,
        sort_by_value=True
    )
    ax.set_title(condition)

# Remove any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


In [ ]:
#Identify Markers - There are Two ways 
#scCellFie runs a TF-IDF approach, as implemented in the tool SoupX in R. We define a express_cut=5*np.log(2) to define cells expressing each metabolic task. Here, a value of 5*np.log(2) indicates that all reactions in the task are over the threshold value of the determinant gene (or “active”). This condition is often rare in single-cell data due to data sparsity (rarely all reactions in a task are over their determinant gene’s threshold), but it is a good approach for identifying markers.
mrks = sccellfie.external.quick_markers(adata.metabolic_tasks,
                                        cluster_key=cell_group,
                                        n_markers=20,
                                        express_cut=5*np.log(2))

In [ ]:
mrks.head()

In [ ]:
#By looking at the distributions given each task TF-IDF scores, we can do some filtering to ensure more specific tasks.
mrks['tf_idf'].hist() 

In [ ]:
scatter = plt.scatter(mrks['tf'], mrks['idf'], c=mrks['tf_idf'], cmap='YlGnBu')
plt.colorbar(scatter)
plt.xlabel('TF')
plt.ylabel('IDF')

In [ ]:
#scCellFie includes a filtering function that fits a hyperbolic curve, while the user can define some thresholds based on the tf-idf scores, and the ratio between the top and second best cell clusters (tf_ratio)
x_col = 'tf'
y_col = 'idf'
df = mrks
tfidf_threshold = 0.3
tf_ratio = 1.2

# Visualization
plt.figure(figsize=(6, 6))

# Plot all points
plt.scatter(df[x_col], df[y_col], alpha=0.5, c='lightgray', label='All points')

# Plot selected points
filtered_mrks, curve = sccellfie.external.filter_tfidf_markers(df, tf_col=x_col, idf_col=y_col, tfidf_threshold=tfidf_threshold, tf_ratio=tf_ratio)

plt.scatter(filtered_mrks[x_col], filtered_mrks[y_col], c='red', label='Selected points')
plt.plot(*curve, label='Fit curve')

plt.xlabel('TF')
plt.ylabel('IDF')
plt.legend()

In [ ]:
tf_idf_mrks = filtered_mrks['gene'].unique().tolist()
len(tf_idf_mrks)

In [ ]:
#We can plot the defining metabolic tasks?
plt.rcParams['figure.figsize'] = (3,3)
plt.rcParams['font.size'] = 10

sc.pl.embedding(adata.metabolic_tasks,
                color=[cell_group] + tf_idf_mrks,
                ncols=3,
                palette=palette,
                frameon=False,
                basis='X_wnn.umap_cc',
                wspace=0.7,
                title=["\n".join(textwrap.wrap(t, width=60)) for t in [cell_group] + tf_idf_mrks],
                cmap='OrRd'
               )

In [ ]:
#A different method to identify metabolism marker pathways
#Similarly, we can use another approach for identifying metabolic markers. In this case, a logistic regression implemented in Scanpy.
method = 'logreg'
sc.tl.rank_genes_groups(adata.metabolic_tasks, cell_group, method=method,
                        use_raw=False, key_added = method)

sc.pl.rank_genes_groups(adata.metabolic_tasks, n_genes=20, sharey=True, key=method, fontsize=4)

In [ ]:
scanpy_df = sc.get.rank_genes_groups_df(adata.metabolic_tasks,
                                          key=method,
                                          group=None)

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata.metabolic_tasks, n_genes=1, groupby=cell_group,
                                key=method, use_raw=False, standard_scale='var',
                               )

In [ ]:
cell_group2 = "FinestCellType"

sc.pl.rank_genes_groups_dotplot(adata.metabolic_tasks, n_genes=1, groupby=cell_group2,
                                key=method, use_raw=False, standard_scale='var',
                               )

In [ ]:
#We can perform Differential expression analysis 
adata.obs.InfectionStatus.unique()

In [ ]:
#Diff regulation part 2 
condition_key = 'InfectionStatus'
reference = 'Naive'

# Get list of experimental conditions excluding "Naive"
experimental_conditions = adata.obs[condition_key].cat.categories
experimental_conditions = [cond for cond in experimental_conditions if cond != reference]

# Create contrast pairs 
contrasts = [(reference, cond) for cond in experimental_conditions]


In [ ]:
import pandas as pd
from statsmodels.stats.multitest import multipletests

all_results = []

for cell_type in adata.obs[cell_group].unique():
    for cond1, cond2 in contrasts:
        dma = sccellfie.stats.scanpy_differential_analysis(adata.metabolic_tasks,
                                                           cell_type=cell_type,
                                                           cell_type_key=cell_group,
                                                           condition_key=condition_key,
                                                           min_cells=20,
                                                           condition_pairs=[(cond1, cond2)])

        if dma is not None and not dma.empty:
            dma['cell_type'] = cell_type
            dma['contrast'] = f"{cond2}_vs_{cond1}"
            all_results.append(dma)

# Combine into single dataframe
dma_combined = pd.concat(all_results, ignore_index=True)


In [ ]:
#Adjust P values globally for the multiple corrections
dma_combined['pval_adj'] = multipletests(dma_combined['p_value'], method='fdr_bh')[1]


In [ ]:
print(dma_combined.columns.tolist())

In [ ]:
#Filter significant results 
# Thresholds
pval_threshold = 0.05
cohen_threshold = .5
logfc_threshold = 0.585 #is equal to 1.5 FC

sig_results = dma_combined[
    (dma_combined['pval_adj'] < pval_threshold) &
    (dma_combined['cohens_d'].abs() > cohen_threshold) &
    (dma_combined['log2FC'].abs() > logfc_threshold)
]


In [ ]:
print(sig_results.columns.tolist())

In [ ]:
# Target cell types
target_cells = ["Enterocytes", "Stem Cells", "Transit Amplifying Cells", "ILC2s"]

# Filter for significant features in target cells vs Naive
sig_naive = sig_results[
    (sig_results['cell_type'].isin(target_cells)) 
]

# Select relevant columns and drop duplicates if any
sig_table = sig_naive[['feature', 'contrast', 'cell_type']].drop_duplicates()

# Show the table (or print, or save)
print(sig_table)

# Optionally, group by condition/cell type to see counts
counts = sig_table.groupby(['contrast', 'cell_type']).size().reset_index(name='n_pathways')
print(counts)

In [ ]:
#Visualize how many differentially expressed pathways we observe
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assume counts_full from before (with all combos and zero-filled)
# If you don't have counts_full yet, create it as:

all_contrasts = sig_table['contrast'].unique()
all_cells = sig_table['cell_type'].unique()

full_index = pd.MultiIndex.from_product([all_cells, all_contrasts], names=['cell_type', 'contrast'])
counts = sig_table.groupby(['cell_type', 'contrast']).size().reset_index(name='n_pathways')
counts_full = counts.set_index(['cell_type', 'contrast']).reindex(full_index, fill_value=0).reset_index()

# Plot 
plt.figure(figsize=(12,6))
sns.barplot(data=counts_full, x='cell_type', y='n_pathways', hue='contrast', palette='tab10')

plt.xticks(rotation=45, ha='right')
plt.ylabel('Number of Significant Pathways')
plt.xlabel('Cell Type')
plt.title('Significant Metabolic Pathways by Cell Type and Condition')
plt.legend(title='Condition', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()



In [ ]:
contrast_of_interest = "Nippostrongylus_vs_Naive"
cell_of_interest = "Enterocytes"

# Filter significant features based on your thresholds
sig = dma_combined[
    (dma_combined['contrast'] == contrast_of_interest) &
    (dma_combined['cell_type'] == cell_of_interest) &
    (dma_combined['pval_adj'] < pval_threshold) &
    (dma_combined['cohens_d'].abs() > cohen_threshold) &
    (dma_combined['log2FC'].abs() > logfc_threshold)
].set_index('feature')

sig_features = sig.sort_values('log2FC').index.tolist()

print(f"Found {len(sig_features)} significant metabolic tasks for {cell_of_interest} in {contrast_of_interest}")
print(sig_features)


In [ ]:
dma_combined.head


In [ ]:
if len(sig_features) > 0:
    width = min(18, 1.5 + 0.5 * len(sig_features))
    fig, ax = sccellfie.plotting.create_comparative_violin(
        adata=adata.metabolic_tasks,
        significant_features=sig_features,
        group1='Naive',
        group2='Nippostrongylus',
        condition_key=condition_key,      
        cell_type_key=cell_group,        
        celltype=cell_of_interest,
        xlabel='',
        ylabel='Metabolic Score',
        palette=['gray', 'darkgreen'],   
        wrapped_title_length=100,
        figsize=(width, 4),
        fontsize=14,
        title=f'{cell_of_interest} ({contrast_of_interest})',
        tight_layout=False,
        # save=f'Violin-{cell_of_interest}-{contrast_of_interest}.pdf'  
    )
else:
    print(f"No significant features found for {cell_of_interest} in {contrast_of_interest} under current thresholds.")




In [ ]:
#Plot a subset Lipis metabolism results for Nippo for the paper 
import matplotlib as mpl
mpl.rcParams['svg.fonttype'] = 'none' 

sig_features2 = [
    "Acetoacetate synthesis",
    "Arachidonate degradation",
    "Arachidonate synthesis", "Ceramide synthesis", "Linoleate degradation"
]

import seaborn as sns
import matplotlib.pyplot as plt



# Subset the adata to only Enterocytes and selected tasks
# Filter for relevant cells and conditions
cell_mask = adata.metabolic_tasks.obs[cell_group] == cell_of_interest
condition_mask = adata.metabolic_tasks.obs[condition_key].isin(['Naive', 'Nippostrongylus'])

adata_subset = adata.metabolic_tasks[cell_mask & condition_mask, :]

# Extract scores for selected tasks
scores = pd.DataFrame(adata_subset[:, sig_features2].X.toarray(), 
                      columns=sig_features2,
                      index=adata_subset.obs_names)

# Add metadata
scores['condition'] = adata_subset.obs[condition_key].values
scores['cell_id'] = scores.index

# Convert to long format
df_long = pd.melt(scores,
                  id_vars=['cell_id', 'condition'],
                  var_name='Metabolic Task',
                  value_name='Score')

# Plot
plt.figure(figsize=(7, 5))
sns.violinplot(
    data=df_long,
    x='Score',
    y='Metabolic Task',
    hue='condition',
    split=True,
    inner='quartile',
    palette={'Naive': 'lightgrey', 'Nippostrongylus': 'navy'}
)

plt.title(f"{cell_of_interest}: Metabolic Task Activity (Split Violin)")
plt.xlabel("Metabolic Score")
plt.ylabel("")
plt.xticks(rotation=30, ha='right')
plt.legend(title="Condition", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

plt.savefig("/path/to/analysis/directory/scCellfie/Images/split_violin_metabolic_tasks.svg", format="svg", dpi=300)
plt.show()

In [ ]:
import matplotlib as mpl
mpl.rcParams['svg.fonttype'] = 'none'

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sig_features2 = [
    "Acetoacetate synthesis",
    "Arachidonate degradation",
    "Arachidonate synthesis",
    "Ceramide synthesis",
    "Linoleate degradation"
]

# Subset the adata
cell_mask = adata.metabolic_tasks.obs[cell_group] == cell_of_interest
condition_mask = adata.metabolic_tasks.obs[condition_key].isin(['Naive', 'Nippostrongylus'])
adata_subset = adata.metabolic_tasks[cell_mask & condition_mask, :]

# Extract scores
scores = pd.DataFrame(
    adata_subset[:, sig_features2].X.toarray(), 
    columns=sig_features2,
    index=adata_subset.obs_names
)

# Add metadata
scores['condition'] = adata_subset.obs[condition_key].values
scores['cell_id'] = scores.index

# Convert to long format
df_long = pd.melt(
    scores,
    id_vars=['cell_id', 'condition'],
    var_name='Metabolic Task',
    value_name='Score'
)

# Plot
plt.figure(figsize=(8, 5))
sns.violinplot(
    data=df_long,
    x='Score',
    y='Metabolic Task',
    hue='condition',
    inner='quartile',
    palette={'Naive': 'lightgrey', 'Nippostrongylus': 'navy'},
    dodge=True  # default is True, can be explicit
)

plt.title(f"{cell_of_interest}: Metabolic Task Activity")
plt.xlabel("Metabolic Score")
plt.ylabel("")
plt.xticks(rotation=30, ha='right')
plt.legend(title="Condition", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

plt.savefig("/path/to/analysis/directory/scCellfie/Images/split_violin_metabolic_tasks.svg", format="svg", dpi=300)
plt.show()


In [ ]:
contrast_of_interest = "Nippostrongylus_vs_Naive"
cell_of_interest = "Enterocytes"

# Step 1: Get all tasks in the Lipids Metabolism system
task_info = adata.metabolic_tasks.uns['task_info']
lipid_tasks = task_info[task_info['System'].str.upper() == 'LIPIDS METABOLISM']['Task'].str.strip().tolist()
lipid_tasks_set = set(lipid_tasks)

# Step 2: Intersect with significant features from differential analysis
sig_lipid_features = [f for f in sig_features if f in lipid_tasks_set]

print(f"Found {len(sig_lipid_features)} significant Lipids Metabolism tasks for {cell_of_interest} in {contrast_of_interest}")
print(sig_lipid_features)

# Proceed if there are any significant lipid-related tasks
if len(sig_lipid_features) > 0:
    # Step 3: Subset the adata to just Enterocytes under Naive or Nippostrongylus
    cell_mask = adata.metabolic_tasks.obs[cell_group] == cell_of_interest
    condition_mask = adata.metabolic_tasks.obs[condition_key].isin(['Naive', 'Nippostrongylus'])
    adata_subset = adata.metabolic_tasks[cell_mask & condition_mask, :]

    # Extract scores for the lipid tasks
    scores = pd.DataFrame(
        adata_subset[:, sig_lipid_features].X.toarray(),
        columns=sig_lipid_features,
        index=adata_subset.obs_names
    )

    # Add metadata
    scores['condition'] = adata_subset.obs[condition_key].values
    scores['cell_id'] = scores.index

    # Step 6: Melt to long format
    df_long = pd.melt(
        scores,
        id_vars=['cell_id', 'condition'],
        var_name='Metabolic Task',
        value_name='Score'
    )


    mpl.rcParams['svg.fonttype'] = 'none'

    fig, ax = plt.subplots(figsize=(min(2 + 0.4 * len(sig_lipid_features), 24), 3.0))
    sns.violinplot(
    data=df_long,
    x='Metabolic Task',   # ⬅️ now on x-axis
    y='Score',            # ⬅️ now on y-axis
    hue='condition',
    inner='box',
    palette={'Naive': 'lightgrey', 'Nippostrongylus': '#ECB364'},
    dodge=True,
    linewidth=1,
    width=0.8,
    ax=ax
    )

    for collection in ax.collections:
        if isinstance(collection, mpl.collections.PolyCollection):
            collection.set_edgecolor('black')
            collection.set_linewidth(1)
        
    #plt.title(f"{cell_of_interest}: Lipids Metabolism Task Activity\n{contrast_of_interest}")
    plt.xlabel("")
    plt.ylabel("Metabolic Score")
    plt.xticks(rotation=30, ha='right')
    plt.legend(title="Condition", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig("/path/to/analysis/directory/scCellfie/Images/split_violin_metabolic_tasks_Nippo_lipids_Sig.svg", format="svg", dpi=300)
    plt.show()

else:
    print("No significant lipid metabolism tasks found under current thresholds.")




In [ ]:
fig, ax = plt.subplots(figsize=(min(2 + 0.6 * len(sig_lipid_features), 30), 3.0))  # Wider, not taller

flierprops = dict(marker='o', markersize=2, markerfacecolor='none', markeredgecolor='black', markeredgewidth=0.5)

sns.boxplot(
    data=df_long,
    x='Metabolic Task',
    y='Score',
    hue='condition',
    palette={'Naive': 'lightgrey', 'Nippostrongylus': '#ECB364'},
    dodge=True,
    linewidth=1,
    width=0.8,
    flierprops=flierprops,
    ax=ax
)

# Force black outlines on boxes and whiskers
for line in ax.lines:
    line.set_color('black')
    line.set_linewidth(1)

for patch in ax.artists:
    patch.set_edgecolor('black')
    patch.set_linewidth(1)

# Add grid lines
ax.yaxis.grid(True, which='major', linestyle='--', linewidth=0.5, color='gray')
ax.xaxis.grid(False)

# Axis labels and formatting
plt.xlabel("")
plt.ylabel("Metabolic Score")
plt.xticks(rotation=30, ha='right')
plt.legend(title="Condition", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# Save the plot
#Figure 5B
plt.savefig(
    "/path/to/analysis/directory/scCellfie/Images/split_boxplot_metabolic_tasks_Nippo_lipids_Sig.svg",
    format="svg", dpi=300
)
plt.show()

In [ ]:
dma_filtered = dma_combined[
    (dma_combined['contrast'] == contrast_of_interest) &
    (dma_combined['cell_type'] == cell_of_interest)
]

# Filter to only the lipid metabolism tasks that you plotted
dma_plot_tasks = dma_filtered[dma_filtered['feature'].isin(sig_lipid_features)]

# Select and sort relevant columns
logfc_table = dma_plot_tasks[['feature', 'log2FC', 'pval_adj']].sort_values('log2FC', ascending=False)

print(logfc_table)
print(dma_combined)

In [ ]:
#Create a Heatmap of selected tasks to ask if these tasks are broadly upregulated in cell types during Nippo infection

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Define custom colormap
custom_cmap = LinearSegmentedColormap.from_list(
    "navy_to_red", ["navy", "lightgrey", "darkred"]
)

# Define task list
tasks_of_interest = [
    'Arachidonate synthesis', 
    'Palmitate degradation', 
    'Elaidate degradation', 
    'cis-vaccenic acid degradation', 
    'Linoleate degradation'
]

# Filter for contrast and tasks of interest across all cell types
subset = dma_combined[
    (dma_combined['contrast'] == "Nippostrongylus_vs_Naive") &
    (dma_combined['feature'].isin(tasks_of_interest))
]

# Pivot to heatmap format: tasks (rows) x cell types (columns)
heatmap_data = subset.pivot(index='feature', columns='cell_type', values='median_diff')

# Optional: enforce row order
heatmap_data = heatmap_data.loc[tasks_of_interest]

# Plot heatmap
plt.figure(figsize=(10, 3))
sns.heatmap(
    heatmap_data,
    cmap=custom_cmap,
    center=0,
    linewidths=0.5,
    linecolor='grey',
    annot=False,
    fmt=".2f",
    cbar_kws={"label": "Median Diff"}
)
plt.xlabel(None)
plt.ylabel(None)
#plt.title("Lipid Task Activity (Log2FC) across Cell Types")
plt.tight_layout()
plt.savefig(
    "/path/to/analysis/directory/scCellfie/Images/Heatmap_metabolic_tasks_Nippo_lipids_Sig.svg",
    format="svg", dpi=300
)
plt.show()





In [ ]:
# Add the finecell type data to the differntial analysis results by creating a mapping
coarse_to_fine = (
    adata.obs.groupby(cell_group)["FineCellType"]
    .apply(lambda x: sorted(x.unique()))
)

expanded_results = []

for _, row in dma_combined.iterrows():
    coarse_type = row["cell_type"]

    # If it's Enterocytes, split to fine cell types
    if coarse_type == "Enterocytes":
        fine_types = coarse_to_fine.get(coarse_type, [None])
        for fine in fine_types:
            new_row = row.copy()
            new_row["fine_cell_type"] = fine
            new_row["cell_type_plot"] = f"{fine}"  # label x-axis by fine type only
            expanded_results.append(new_row)

    # Otherwise, keep as single coarse type
    else:
        new_row = row.copy()
        new_row["fine_cell_type"] = coarse_type  # keep same label
        new_row["cell_type_plot"] = coarse_type
        expanded_results.append(new_row)

dma_expanded = pd.DataFrame(expanded_results)


In [ ]:
for ct in dma_expanded["cell_type_plot"].unique():
    print(ct)

In [ ]:
print(dma_expanded)

desired_order = [
     "Early Enterocyte", "Middle Enterocyte", "Late Enterocyte",
    "Goblet Cells","Enteroendocrine Cells",  "Paneth Cells",
    "Transit Amplifying Cells", "Tuft Cells", "Stem Cells",
    "CD8 ab T Cells", "Effector CD8 ab T Cells",
    "gd T Cells", "Cycling T Cells", "CD4 ab T Cells",
    "Effector CD4 ab T Cells", "NKT Cells", "NK Cells",
    "ILC1s", "ILC2s", "ILC3s", 
    "B Cells", "GC B Cells", "Plasma Cells",
     "cDC2s",  "Macrophages",
    "Monocytes",  "Plasmacytoid DCs",
    "Lymphatic Endothelial Cells", 
    "Fibroblasts", "Smooth Muscle Cells"
]

dma_expanded["cell_type_plot"] = pd.Categorical(
    dma_expanded["cell_type_plot"],
    categories=desired_order,
    ordered=True
)
tasks_of_interest = [
    'Arachidonate synthesis', 
    'Palmitate degradation', 
    'Elaidate degradation', 
    'cis-vaccenic acid degradation', 
    'Linoleate degradation'
]
subset = dma_expanded[
    (dma_expanded['contrast'] == "Nippostrongylus_vs_Naive") &
    (dma_expanded['feature'].isin(tasks_of_interest))
]

heatmap_data = subset.pivot(
    index='feature',
    columns='cell_type_plot',
    values='median_diff'
)

# Reorder columns to match biological order
heatmap_data = heatmap_data.reindex(columns=desired_order)


plt.figure(figsize=(12, 4.5))
sns.heatmap(
    heatmap_data,
    cmap=custom_cmap,
    center=0,
    linewidths=0.5,
    linecolor='grey',
    cbar_kws={"label": "Median Diff"}
)
plt.tight_layout()
plt.savefig(
    "/path/to/analysis/directory/scCellfie/Images/Heatmap_metabolic_tasks_Nippo_lipids_Sig_EnterocyteSplit.svg",
    format="svg", dpi=300
)
plt.show()
#Figure 5C